In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.nn.init import kaiming_uniform_
from torch.distributions import Normal

In [2]:
torch.normal(mean=torch.arange(1., 6.), std=torch.arange(1., 6.))

tensor([0.7506, 1.1183, 5.6112, 3.3301, 2.9043])

In [3]:
import numpy as np
import gym
from tqdm import tqdm
import random as rand
from itertools import count

In [4]:
class ReplayMemory():
    def __init__(self,capacity):   
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count%self.capacity] = experience
        self.push_count+=1
    
    def sample(self, batch_size):
        return rand.sample(self.memory,batch_size)
    
    def can_provide_sample(self, batch_size):
        return len(self.memory)>=batch_size
    
    def update_td_error(self, sampled_experiences):
        for sampled_idx,sampled_exp in enumerate(sampled_experiences):
            for mem_idx, mem_exp in enumerate(self.memory):
                if mem_exp.timestep == sampled_exp.timestep:
                    self.memory[mem_idx] = sampled_exp #update memory
                    break
        
    def get_memory_values(self):
        return self.memory    

In [5]:
def extract_tensors(experiences):
    #print(".....................................................")
    #print(experiences)
    batch = Xp(*zip(*experiences))
    state = np.stack(batch.state) #stack
    action = np.stack(batch.action)
    next_state = np.stack(batch.next_state)
    reward = np.stack(batch.reward)
    done = np.stack(batch.done)
    abs_td_error = np.stack(batch.abs_td_error)
    timestep = np.stack(batch.timestep)
    return state,action,next_state,reward,done,abs_td_error,timestep

In [6]:
def rebuild_experiences(state, action, next_state, reward, done, abs_error, timestep):
    exp_list = []
    for idx_ in range(len(state)):
        exp_list.append(\
                        Xp(state[idx_], action[idx_], next_state[idx_], reward[idx_],\
                           done[idx_], abs_error[idx_], timestep[idx_]))
    return exp_list

In [7]:
from collections import namedtuple
Xp = namedtuple('Experience',
                        ('state', 'action', 'next_state', 'reward', 'done', 'abs_td_error','timestep'))
Xp_points = Xp(5,6,7,8,9,10,11)
Xp_points

Experience(state=5, action=6, next_state=7, reward=8, done=9, abs_td_error=10, timestep=11)

In [8]:
def prioritize_samples(experience_samples, alpha, beta):
    state,action,next_state,reward,done,abs_td_error,timesteps = extract_tensors(experience_samples)
    #rank based
    #('state', 'action', 'next_state', 'reward', 'done', 'abs_td_error','timestep')
    abs_td_error  = np.expand_dims(abs_td_error, axis=1)
    abs_td_error = torch.tensor(abs_td_error)
    abs_td_error, indices_ = abs_td_error.sort(0, descending=True)#big to small
    indices = np.arange(1, len(abs_td_error)+1)
    priorities = 1.0/indices
    priorities = priorities**alpha#scale by alpha
    priorities = np.expand_dims(priorities, axis=1)
    probabilities = priorities/np.sum(priorities, axis=0)#sums up to 1(or 0.999999)
    assert np.isclose(probabilities.sum(), 1.0)#ensures probs add up to 1
    
    number_of_samples  = len(probabilities)
    weight_importance_ = number_of_samples*probabilities
    weight_importance_ = weight_importance_**-beta
    weight_importance_max = np.max(weight_importance_)
    weight_importance_scaled = weight_importance_/weight_importance_max
    return weight_importance_scaled, indices_ #return weight important samples, return indices for re_arranging sampled experiences

In [9]:
class linearApproximator_FCGSAP(nn.Module):
    def __init__(self,state_shape,outputs,hidden_dims=(32,32), log_entropy_lr =0.0001,\
                log_std_dev_min=-20, log_std_dev_max= 2, epsilon = 1e-6):
        super(linearApproximator_FCGSAP, self).__init__()
        self.input_size = state_shape
        self.out = outputs
        self.log_std_dev_min = log_std_dev_min
        self.log_std_dev_max = log_std_dev_max
        self.epsilon = epsilon
        self.device = torch.device("cuda" if torch.cuda.is_available()\
                                   else "cpu")
        
        self.fc1  = nn.Linear(self.input_size,hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            hidden_layer = nn.Linear(\
                                hidden_dims[i], hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        
        self.output_layer_distribution  = nn.Linear(hidden_dims[-1],self.out)
        self.output_layer_mean = nn.Linear(hidden_dims[-1],self.out)
        
        self.target_entropy = -float(self.out)
        #self.target_entropy = self.target_entropy.float() 
        #according to the eq, log alpha is a learnable parameter
        self.log_alpha = torch.zeros(1,\
                                     requires_grad=True,\
                                     device = self.device)
        self.log_alpha_optimizer = torch.optim.Adam([self.log_alpha],\
                                                    lr=log_entropy_lr)
                                     
        self.to(self.device)
        
    def forward(self, state_shape):
        if not isinstance(state_shape, torch.Tensor):
            state_shape = torch.tensor(state_shape, dtype=torch.float32)
        state_shape = state_shape.to(self.device)
        x = self.fc1(state_shape)
        x = F.relu(x)
        
        for hidden_layer in self.hidden_layers:
            x = F.relu(hidden_layer(x))
        
        distribution = self.output_layer_distribution(x)#logits, preferences of actions
        mean   = self.output_layer_mean(x)
        distribution = torch.clamp(distribution, self.log_std_dev_min, self.log_std_dev_max)
        return mean, distribution
        
    def full_pass(self, state):
        mean, distribution = self.forward(state)
        pi_s = Normal(mean, distribution.exp())
        pre_sampled_actions = pi_s.rsample()
        sampled_actions = torch.tanh(pre_sampled_actions)#scale actions between -1 and 1
        #we also rescale our logprobs to match action space
        log_probs = pi_s.log_prob(pre_sampled_actions) - \
                                    torch.log((1 - sampled_actions.pow(2)).clamp(0,1) + \
                                                self.epsilon)
        log_probs = log_probs.sum(dim=1, keepdim=True)
        return sampled_actions, log_probs, mean

In [10]:
class linearApproximator_FCQV(nn.Module):#Q value of state action pair
    def __init__(self,state_shape,action_outputs_size,hidden_dims=(32,32)):
        super(linearApproximator_FCQV, self).__init__()
        self.input_size = state_shape
        self.action_outputs_size = action_outputs_size
        self.device = torch.device("cuda" if torch.cuda.is_available()\
                                   else "cpu")
        
        self.fc1  = nn.Linear(self.input_size,hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            hidden_input_layer = hidden_dims[i]
            if i == 0:
                hidden_input_layer += self.action_outputs_size #increased to account for size/number of actions
            hidden_layer = nn.Linear(\
                                hidden_input_layer, hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        
        self.output_layer  = nn.Linear(hidden_dims[-1],1)
        self.to(self.device)
        
    def forward(self, state_shape, action_shape):
        if not isinstance(state_shape, torch.Tensor):
            state_shape = torch.tensor(state_shape, dtype=torch.float32).to(self.device)
        if not isinstance(action_shape, torch.Tensor):
            action_shape = torch.tensor(action_shape, dtype=torch.float32).to(self.device)
                    
        x = self.fc1(state_shape)
        x = F.relu(x)
        
        for idx, hidden_layer in enumerate(self.hidden_layers):
            if idx == 0:
                x = torch.cat((x, action_shape), dim=1)
            x = F.relu(hidden_layer(x))
        
        q_value = self.output_layer(x)#logits, preferences of actions
        return q_value

In [11]:
def update_networks(online_q_network_a, online_q_network_b,\
                    offline_q_network_a, offline_q_network_b, tau):
        
    for target_weights, online_weights in zip(offline_q_network_a.parameters(), online_q_network_a.parameters()):
        target_weight_update = (1.0 - tau)*target_weights.data
        online_weight_update = tau*online_weights.data
        sum_up = target_weight_update + online_weight_update
        target_weights.data.copy_(sum_up)
        
    for target_weights, online_weights in zip(offline_q_network_b.parameters(), online_q_network_b.parameters()):
        target_weight_update = (1.0 - tau)*target_weights.data
        online_weight_update = tau*online_weights.data
        sum_up = target_weight_update + online_weight_update
        target_weights.data.copy_(sum_up)

    return offline_q_network_a, offline_q_network_b

In [21]:
def update_online_model(experience_samples,\
                        online_policy_network, online_q_network_a, online_q_network_b,\
                        online_policy_optimizer, online_q_optimizer_a, online_q_optimizer_b,\
                        offline_q_network_a, offline_q_network_b,\
                        gamma, weighted_importance, indices):
    
    states, actions, next_states, rewards, done, td_errors, timesteps = extract_tensors(experience_samples)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #rearrange first
    indices = np.array(indices)#have to be same size as batch size(already taken care of)
    indices = np.squeeze(indices)
    states = states[indices,:]
    states = np.squeeze(states)
    actions = actions[indices]
    next_states = next_states[indices,:]
    next_states = np.squeeze(next_states)
    rewards = rewards[indices]
    done = done[indices]
    td_errors = td_errors[indices]
    timesteps = timesteps[indices]    
    
    states = torch.tensor(states).float().to(device)
    actions = torch.tensor(actions)
    actions = actions.float().to(device)
    next_states=torch.tensor(next_states).float().to(device)
    rewards = torch.tensor(rewards).float().to(device)
    rewards = rewards.unsqueeze(1)
    done = torch.tensor(done).float().to(device)
    done = done.unsqueeze(1)
    weighted_importance = torch.tensor(weighted_importance).float().to(device)
    
    
    current_actions,log_pi, _ = online_policy_network.full_pass(states)
    target_alpha = (log_pi +\
                    online_policy_network.target_entropy).detach()
    target_alpha_loss = -(online_policy_network.log_alpha *\
                         target_alpha).mean()
    online_policy_network.log_alpha_optimizer.zero_grad()
    target_alpha_loss.backward()
    online_policy_network.log_alpha_optimizer.step()
    optimized_alpha = online_policy_network.log_alpha.exp()
    
    
    max_q_sa_online_a = online_q_network_a(states, current_actions.detach())
    max_q_sa_online_b = online_q_network_b(states, current_actions.detach())
    max_q_sa_online_a*=weighted_importance
    max_q_sa_online_b*=weighted_importance
    max_q_online__ = torch.min(max_q_sa_online_a, max_q_sa_online_b)
    max_q_online__*=(1 - done)
    #print("max_q_online__", max_q_online__)
    predicted_online_action_policy,\
                log_pi_ns, _ = online_policy_network.full_pass(next_states)

    policy_loss = (optimized_alpha.detach() * log_pi - max_q_online__.detach()).mean()#policy loss
    
    max_q_sa_offline_a = offline_q_network_a(next_states, predicted_online_action_policy)
    max_q_sa_offline_b = offline_q_network_b(next_states, predicted_online_action_policy)
    max_q_sa_offline = torch.min(max_q_sa_offline_a, max_q_sa_offline_b)
    TWIN_target = max_q_sa_offline
    TWIN_target*=weighted_importance
    
    TWIN_target = TWIN_target.detach()
    TWIN_target *=(1 - done)
    TWIN_target = TWIN_target - optimized_alpha * log_pi_ns
    TWIN_target = rewards + gamma*TWIN_target
    
    #abs_a = abs(TWIN_target.detach().cpu().numpy() - max_q_sa_online_a.detach().cpu().numpy())
    #abs_b = abs(TWIN_target.detach().cpu().numpy() - max_q_sa_online_b.detach().cpu().numpy())
    #ovr_abs_update = (abs_a + abs_b)/2 #we get the mean(not done)
    
    TWIN_target = TWIN_target.detach()
    loss_func = torch.nn.SmoothL1Loss()
    
    
    q_sa_online_a = online_q_network_a(states, actions)
    q_sa_online_b = online_q_network_b(states, actions)
    
    abs_a = abs(TWIN_target.detach().cpu().numpy() - max_q_sa_online_a.detach().cpu().numpy())
    abs_b = abs(TWIN_target.detach().cpu().numpy() - max_q_sa_online_b.detach().cpu().numpy())
    ovr_abs_update = (abs_a + abs_b)/2 #we get the mean(not done)
    
    
    
    q_online_value_loss_a = loss_func(q_sa_online_a,\
                                             TWIN_target)
    q_online_value_loss_b = loss_func(q_sa_online_b,\
                                             TWIN_target)
    online_q_optimizer_a.zero_grad()
    q_online_value_loss_a.backward()
    online_q_optimizer_a.step()
    online_q_optimizer_b.zero_grad()
    q_online_value_loss_b.backward()
    online_q_optimizer_b.step()
    
    #One of the difference between SAC and TD3 is SAC only uses one online policy
    #there is also no delay in policy updates in SAC
    #policy_loss = -(pre_optimized_alpha * log_pi - max_q_online__).mean()
    online_policy_optimizer.zero_grad()
    policy_loss.backward()
    online_policy_optimizer.step()
        
    states, actions, next_states, rewards, done, td_errors, timesteps = extract_tensors(experience_samples)
    experiences_rebuilded = rebuild_experiences(states, actions, next_states, rewards, done, ovr_abs_update, timesteps)
    return experiences_rebuilded

In [22]:
def query_error(online_policy_network, offline_q_network_a, offline_q_network_b,\
                online_q_network_a, online_q_network_b, state, action, next_state, reward, gamma):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    state = torch.tensor(state).float().to(device)
    state = state.unsqueeze(0)
    next_state = torch.tensor(next_state).float().to(device)
    next_state = next_state.unsqueeze(0)
    
    alpha = online_policy_network.log_alpha.exp()
    
    ns_actions,log_pi_ns, _ = online_policy_network.full_pass(next_state)
    q_target_next_states_action_a = offline_q_network_a(next_state,\
                                                    ns_actions.detach())
    q_target_next_states_action_b = offline_q_network_b(next_state,\
                                                    ns_actions.detach())
    
    
    TWIN_target = torch.min(q_target_next_states_action_a, q_target_next_states_action_b)
    TWIN_target = TWIN_target - alpha * log_pi_ns
    TWIN_target = reward + (gamma*TWIN_target.detach())
    
    
    current_action,_, _ = online_policy_network.full_pass(state)
    #print("state: ", state.shape)
    #print("action: ", action.shape)
    action = np.expand_dims(action, axis=0)
    #print("action: ", action.shape)
    q_online_state_action_val_a = online_q_network_a(state, action)
    q_online_state_action_val_b = online_q_network_b(state, action)
    
    abs_a = abs(TWIN_target - q_online_state_action_val_a)
    abs_b = abs(TWIN_target - q_online_state_action_val_b)
    abs_stack = (abs_a + abs_b)/2
    #print("abs querry error stacked: ", abs_stack.shape)
    ovr_abs_update = abs_stack
    #print("abs querry error mean: ", ovr_abs_update.shape)
    return ovr_abs_update.detach().cpu().numpy()

In [23]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False
    return model

In [24]:
def select_action(state, online_policy_network):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    state = torch.tensor(state).float().to(device)
    state = state.unsqueeze(0)
    with torch.no_grad():
        actions,log_pi_, _ = online_policy_network.full_pass(state)
        actions = actions.cpu().detach()
        actions = actions.data.numpy().squeeze()
    return actions

In [25]:
def SAC_PER(env,
         gamma=0.99,
         alpha_pr=0.6,
         beta_pr=0.3,
         memory_size = 50000,
         tau = 0.01,
         offline_update = 1000,
         min_sample_size=200,
         batch_size = 64,
         n_ep=2000,
         max_steps = 100000
         ):
    
    
    observation_space = len(env.reset())
    action_space_high, action_space_low = env.action_space.high, env.action_space.low
    n_actions = len(action_space_high)
    online_policy_network = linearApproximator_FCGSAP(observation_space,n_actions,\
                                     hidden_dims=(128,64))
    online_q_network_a = linearApproximator_FCQV(observation_space,\
                                     n_actions,hidden_dims=(128,64))
    online_q_network_b = linearApproximator_FCQV(observation_space,\
                                     n_actions,hidden_dims=(128,64))
    
    offline_q_network_a = linearApproximator_FCQV(observation_space,\
                                     n_actions,hidden_dims=(128,64))
    offline_q_network_b = linearApproximator_FCQV(observation_space,\
                                     n_actions,hidden_dims=(128,64))
    

    offline_q_network_a.eval()
    offline_q_network_a = freeze_model(offline_q_network_a)
    offline_q_network_b.eval()
    offline_q_network_b = freeze_model(offline_q_network_b)
    
    online_policy_optimizer    = torch.optim.Adam(online_policy_network.parameters(),lr=0.0008)
    online_q_optimizer_a = torch.optim.Adam(online_q_network_a.parameters(),lr=0.0008)
    online_q_optimizer_b = torch.optim.Adam(online_q_network_b.parameters(),lr=0.0008)
    
    memory = ReplayMemory(memory_size)
    
    t_step = 0 #important
    reward_per_ep = []
    
    
    for e in tqdm(range(n_ep)):
        state = env.reset()
        reward_accumulated = 0
        
        while True:
            env.render()
            action = select_action(state, online_policy_network)
            
            next_state, reward, done, info = env.step(action)
            td_error = query_error(online_policy_network, offline_q_network_a, offline_q_network_b,\
                online_q_network_a, online_q_network_b, state, action, next_state, reward, gamma)
            #print("td error: ", td_error.shape)
            td_error = np.squeeze(td_error, axis = 0)
            #print("td error: ", td_error.shape)
            reward_accumulated+=reward
            is_truncated = 'TimeLimit.truncated' in info and\
                                info['TimeLimit.truncated']
            is_failure = done and not is_truncated
           
            memory.push(Xp(state, action, next_state, reward, is_failure, td_error, t_step))
            state = next_state
            t_step+=1
            if memory.can_provide_sample(min_sample_size):
                experience_samples = memory.sample(batch_size)
                weighted_importance, indices = prioritize_samples(experience_samples, alpha_pr, beta_pr)
                rebuilded_exp = update_online_model(experience_samples,\
                        online_policy_network, online_q_network_a, online_q_network_b,\
                        online_policy_optimizer, online_q_optimizer_a, online_q_optimizer_b,\
                        offline_q_network_a, offline_q_network_b,\
                        gamma, weighted_importance, indices)
                memory.update_td_error(rebuilded_exp)
                
            if t_step%offline_update == 0:
                offline_q_network_a, offline_q_network_b = update_networks(online_q_network_a, online_q_network_b,\
                                                                    offline_q_network_a, offline_q_network_b, tau)
            if done == True:
                reward_per_ep.append(reward_accumulated)
                #print(reward_accumulated)
                break
            if t_step > max_steps:
                env.close()
                return reward_per_ep
    env.close()           
    return reward_per_ep

In [26]:
import gym
env = gym.make('BipedalWalker-v3')

In [29]:
env.close()

In [28]:
rewards = SAC_PER(env)  

  0%|          | 0/2000 [00:00<?, ?it/s]

[ 0.31225717  0.89702725 -0.79252076 -0.44635463]
[-0.7432438  -0.8940234  -0.4491917   0.42728582]
[-0.79932886 -0.854982   -0.9551391  -0.9587367 ]
[-0.82561314 -0.05194125  0.81717324 -0.26043847]
[ 0.22441521 -0.37230337 -0.00641009 -0.79782015]
[ 0.91675013  0.46006736 -0.6129836   0.82974625]
[-0.26337644 -0.0582804   0.31584558 -0.8430401 ]
[-0.59974164  0.7967043  -0.48568317 -0.76293117]
[-0.09907963 -0.8899228   0.64167094  0.8945864 ]
[ 0.29777417 -0.16318825 -0.6858963   0.87014484]
[ 0.331387   -0.78806055 -0.2505248  -0.6301668 ]
[ 0.6513088  -0.5614964  -0.62408084  0.8819127 ]
[ 0.26891983  0.75908196 -0.4508458  -0.26026094]
[ 0.25109038 -0.23522618  0.435487   -0.00044958]
[ 0.748021   0.5204329 -0.8629143  0.7069999]
[ 0.32604787  0.6999279   0.9269354  -0.6119741 ]
[ 0.62071466 -0.91474396 -0.30140376 -0.33809263]
[ 0.92291385  0.0800845  -0.82218564  0.05405251]
[ 0.5191721  -0.7326735   0.6305177  -0.66391534]
[-0.48643303  0.85014284  0.74433684 -0.02755189]
[-0.

[ 0.46929437  0.09244566 -0.18194325 -0.6485461 ]
[-0.5646587  -0.60220456  0.71915007  0.50074166]
[-0.4472952   0.7664155   0.71983147 -0.2669641 ]
[ 0.41013837 -0.7216784   0.96333796 -0.59874785]
[ 0.9588553   0.60931253 -0.9262006  -0.8481347 ]
[ 0.553233   -0.9266383  -0.30905488 -0.04399695]
[ 0.70065486 -0.5263101   0.60632    -0.8342071 ]
[-0.25815257  0.31663632 -0.18773615 -0.2542248 ]
[-0.82333195  0.8902515  -0.83909875  0.23362584]
[ 0.36499986  0.08882324 -0.9567853  -0.35637525]
[-0.92543584 -0.2711756   0.25791404  0.27508685]
[-0.83575875 -0.2853313  -0.5918309   0.16523834]
[ 0.164723    0.42776415 -0.76163656 -0.5239291 ]
[ 0.92358285 -0.61245835 -0.43060362  0.04354104]
[ 0.26057747  0.38321474 -0.02818973  0.3944738 ]
[-0.7496264   0.85116357 -0.48021057  0.50318414]
[ 0.0592734   0.18562552 -0.14054944  0.77329016]
[-0.02984556  0.37899756 -0.947459    0.6205896 ]
[-0.6709941   0.20106319 -0.43797758 -0.7609483 ]
[0.22056925 0.21665254 0.35363418 0.21675661]
[-0.

[ 0.20590842 -0.703655   -0.28982306  0.9435683 ]
[-0.19456366 -0.3568477   0.71300113 -0.3749177 ]
[-0.355209   -0.9056148  -0.11369034 -0.30114037]
[ 0.0905792  -0.7572904   0.10344674  0.519364  ]
[ 0.33668172 -0.54940146 -0.84074277 -0.18966275]
[ 0.9145525  -0.9847863   0.69093573 -0.13034359]
[-0.24982554  0.92174727 -0.6842705  -0.7109461 ]
[0.21213782 0.6281527  0.8809123  0.71144617]
[-0.8337972  -0.69043016 -0.56214106  0.80856615]
[-0.53864783 -0.63756895  0.91733706  0.68397605]
[ 0.9072213  -0.7378766  -0.11882979 -0.9301195 ]
[-0.5532026  0.8016565  0.9731082 -0.5740516]
[0.81580055 0.42317155 0.87177235 0.6609159 ]
[ 0.89178807  0.46649218  0.5570138  -0.46686167]
[ 0.4182994   0.9048997   0.10698628 -0.83991706]
[-0.89483696  0.0432418  -0.9387371   0.16126318]
[ 0.96104187 -0.10978534 -0.4578757   0.35400385]
[ 0.12123009 -0.9563756  -0.8571943  -0.34925365]
[0.17102568 0.698953   0.6799754  0.78465015]
[ 0.4353387   0.25051776 -0.34419358  0.1860269 ]
[-0.11612336  0.

[-0.95086837  0.8716552  -0.27714926 -0.97078115]
[-0.9404232  -0.17526583 -0.54847735  0.39705673]
[-0.7862877   0.7976882  -0.09314978  0.03875855]
[-0.10637313  0.22446173  0.04330362 -0.64732707]
[0.54335344 0.41732976 0.83104175 0.20465341]
[-0.54391944 -0.6628015  -0.7060667   0.86104953]
[-0.9606271   0.8073592   0.07819515  0.39050764]
[-0.6194048   0.9184751  -0.8435751  -0.24493454]
[-0.19576168 -0.8865966   0.99435395 -0.17542866]
[-0.18370621 -0.494956   -0.47999105  0.8606797 ]
[-0.67511123 -0.78851825 -0.34006464  0.6463059 ]
[-0.09490444 -0.5910299  -0.22184964  0.1103148 ]
[-0.3876792   0.77409995 -0.3562525  -0.88321835]
[ 0.5180598   0.8473507   0.11077297 -0.82814616]
[-0.7026142  -0.21185648 -0.7549128  -0.44665438]
[ 0.2584338  -0.26324627  0.96502423  0.24921578]
[ 0.59147394  0.8720606   0.81912    -0.34009725]
[-0.1598643  -0.19351168  0.82693243 -0.449664  ]
[ 0.13281022 -0.2704377   0.45894548  0.78836256]
[ 0.28693527  0.9183649  -0.9725562   0.33790186]
[ 0.

...

[ 0.8814738   0.9025409  -0.5661303  -0.72029895]
[-0.0432312   0.43416864  0.30374938  0.7463356 ]
[-0.24275182 -0.3458029  -0.4179728   0.23712023]
[ 0.5301634  -0.9385265  -0.15391517  0.2466422 ]
[-0.8900351  -0.0824773  -0.3187452   0.13196419]
[-0.21822898 -0.4782569  -0.9555384  -0.1083174 ]
[ 0.29668334  0.51676947  0.20514108 -0.73725545]
[0.03891153 0.8741808  0.48954883 0.405291  ]
[ 0.5485058   0.24710722 -0.49260613  0.54367733]
[ 0.6272806   0.59385437 -0.89660716  0.41802198]
[ 0.5922767   0.48602068 -0.90845466  0.11859708]
[ 0.83423114  0.9057782  -0.47467566  0.50771976]
[-0.8111366  -0.45673355  0.04193632 -0.7084329 ]
[-0.1128533  -0.7541508   0.00955142 -0.17616215]
[-0.15334609 -0.91169333 -0.6728413   0.9578382 ]
[ 0.7897248   0.64643335 -0.54549617 -0.64070934]
[-0.06101749 -0.9864154   0.172522   -0.08122513]
[ 0.4406176   0.98406273 -0.53638536  0.7219734 ]
[ 0.05201292 -0.20140794 -0.42176017 -0.54347056]
[0.6795765  0.4801408  0.33898544 0.07671895]
[-0.8326

[-0.89319354 -0.38680017  0.5050089  -0.35372475]
[-0.7785382   0.30216366 -0.07867738  0.43698615]
[-0.39930445  0.8449674  -0.62907296 -0.1398222 ]
[ 0.68923473  0.22059117  0.6261896  -0.44534567]
[-0.87620085 -0.20628907  0.2611731  -0.73787165]
[ 0.5641063  -0.08187351 -0.2446106  -0.32433817]
[-0.32059348 -0.8113762  -0.11125311  0.83053523]
[ 0.7809534   0.6214713  -0.7120398   0.73299515]
[ 0.81435335 -0.8607133  -0.39280957 -0.18713419]
[0.5104477  0.3375167  0.8880453  0.03374292]
[-0.29678458  0.00976466  0.5273079   0.8309325 ]
[ 0.95803267 -0.6041082   0.62384975  0.6860798 ]
[ 0.8820937  -0.797711    0.54576576  0.68667865]
[ 0.364152    0.06774659 -0.5059365  -0.912087  ]
[ 0.00147613  0.56365216  0.3148344  -0.76118076]
[-0.25274694  0.48293063 -0.55293036 -0.60012805]
[ 0.17459527  0.49717864 -0.91990703  0.11216138]
[0.7750177  0.7651678  0.69219714 0.5001782 ]
[0.10900382 0.06372924 0.53850484 0.3597948 ]
[-0.45613047 -0.23310693  0.37158513 -0.03677276]
[-0.5372499 

  0%|          | 5/2000 [01:54<12:42:02, 22.92s/it]

[-0.06910258 -0.5134864  -0.724423    0.9034041 ]


KeyboardInterrupt: 

In [ ]:
from scipy.ndimage.filters import uniform_filter1d
arr = uniform_filter1d(rewards, size=1)

In [ ]:
from matplotlib import pyplot as plt
x = []
y = []
for i,e in enumerate(arr):
    y.append(i)
    x.append(e)

In [ ]:
plt.plot(y, x)

In [ ]:
-np.prod(env.action_space.high.shape)